In [21]:
import pandas as pd
import glob
import pickle
from tqdm import tqdm

In [5]:
../data/output/newssniffer-nytimes/articles-with-headlines/

output_chunk__start-0_end-5.pkl         output_chunk__start-4015_end-6015.pkl
output_chunk__start-10_end-15.pkl       output_chunk__start-5_end-10.pkl
output_chunk__start-15_end-2015.pkl     output_chunk__start-6015_end-8015.pkl
output_chunk__start-2015_end-4015.pkl   output_chunk__start-8015_end-10015.pkl


In [13]:
fs = glob.glob('../data/output/newssniffer-nytimes/articles-with-headlines/*')
all_processed = []
for fname in fs:
    with open(fname, 'rb') as f:
        processed = pickle.load(f)
        all_processed.extend(processed)

In [15]:
all_processed[0]

processed_df = (
    pd.DataFrame(all_processed)
    .drop_duplicates('entry_id')
)

In [18]:
processed_df.iloc[0]['quote_idxes']

,head_span,head_s_idx,quote_chunks,background_chunks
head_ref,,,,
0,"(176, 178)",6,"[(23, (506, 507)), (68, (1323, 1324)), (66, (1...",[]
2,"(832, 834)",42,"[(42, (832, 834))]",[]
3,"(857, 859)",44,"[(46, (941, 942)), (45, (889, 890))]","[(44, (857, 859))]"
5,"(1043, 1045)",52,"[(75, (1400, 1401)), (74, (1400, 1401)), (73, ...",[]


In [27]:
# ! gunzip ../../edit-pathways/data/diffengine-diffs/db/newssniffer-nytimes.db.gz

In [29]:
import sqlite3

In [30]:
db_path = '../../edit-pathways/data/diffengine-diffs/db/newssniffer-nytimes.db'
conn = sqlite3.connect(db_path)
orig_data_entries = pd.read_sql(
    '''SELECT * FROM entryversion 
        WHERE entry_id IN (%s) and version = 0
    ''' % ', '.join(processed_df['entry_id'].astype(str)), con=conn)

In [31]:
data_df = processed_df.merge(
    orig_data_entries[['entry_id', 'version', 'title', 'created', 'url']], 
    right_on=['entry_id', 'version'], 
    left_on=['entry_id', 'version']
)

In [32]:
for idx in tqdm(range(len(data_df))):
    q = (
        data_df
         .iloc[idx]['quote_idxes']
         .loc[lambda df: df['quote_chunks'].str.len() > 0]
        )
    if len(q) > 0:
        s = data_df.iloc[idx]['sent_parse']
        break
        s_html = generate_packet_to_annotate(q, s)

  0%|          | 0/2467 [00:00<?, ?it/s]


In [34]:
q

,head_span,head_s_idx,quote_chunks,background_chunks
head_ref,,,,
0,"(176, 178)",6,"[(23, (506, 507)), (68, (1323, 1324)), (66, (1...",[]
2,"(832, 834)",42,"[(42, (832, 834))]",[]
3,"(857, 859)",44,"[(46, (941, 942)), (45, (889, 890))]","[(44, (857, 859))]"
5,"(1043, 1045)",52,"[(75, (1400, 1401)), (74, (1400, 1401)), (73, ...",[]


In [38]:
def flatten_list_of_lists(l_of_l):
    return [i for s in l_of_l for i in s]

In [42]:
l = list(map(len, s))
comp_mapper = {i:idx for idx, i in enumerate(q.index.tolist())}
source_idx_head_mapper = (
 q
 .assign(
     head=lambda df: df.apply(lambda x: ' '.join(s[x['head_s_idx']][
        x['head_span'][0] - sum(l[:x['head_s_idx']]):
        x['head_span'][1] - sum(l[:x['head_s_idx']])]
    ), axis=1 ))
    ['head'].to_dict()
)


span_df = (
 q
 .assign(q=lambda df: df.apply(lambda x: 
   list(map(lambda y: {'source': x.name, 'sent': y[0], 'span': y[1], 'type': 'quote'}, x['quote_chunks'] )), axis=1))
 .assign(b=lambda df: df.apply(lambda x: 
   list(map(lambda y: {'source': x.name, 'sent': y[0], 'span': y[1], 'type': 'background'}, x['background_chunks'] )), axis=1))
 .apply(lambda x: x['q'] + x['b'], axis=1)
 .pipe(lambda s: pd.DataFrame(flatten_list_of_lists(s.tolist())))
 .sort_values('sent', ascending=False)
 .assign(head=lambda df: df['source'].map(source_idx_head_mapper))
 .assign(source=lambda df: df['source'].map(comp_mapper))
)

In [99]:
q

,head_span,head_s_idx,quote_chunks,background_chunks
head_ref,,,,
0,"(176, 178)",6,"[(23, (506, 507)), (68, (1323, 1324)), (66, (1...",[]
2,"(832, 834)",42,"[(42, (832, 834))]",[]
3,"(857, 859)",44,"[(46, (941, 942)), (45, (889, 890))]","[(44, (857, 859))]"
5,"(1043, 1045)",52,"[(75, (1400, 1401)), (74, (1400, 1401)), (73, ...",[]


In [98]:
(q
 .assign(quote_chunks_with_sent_source=lambda df: df['quote_chunks'].apply(lambda x: list(map(lambda y: (y[0], (y[1][0] - sum(l[:y[0]]), y[1][1] - sum(l[:y[0]]))), x))))
 .assign(sents=lambda df: df['quote_chunks_with_sent_source'].apply(lambda x: list(map(lambda y: s[y[0]], x))))
#  .assign(neg_sents = lambda df: df['quote_chunks_with_sent_source'].apply(lambda x: list(filter(lambda y: y[1][0] < 0, x))).apply(lambda x: list(map(lambda y: s[y[0]], x))))
#  ['neg_sents'][0]
)

,head_span,head_s_idx,quote_chunks,background_chunks,quote_chunks_with_sent_source,sents
head_ref,,,,,,
0,"(176, 178)",6,"[(23, (506, 507)), (68, (1323, 1324)), (66, (1...",[],"[(23, (15, 16)), (68, (-4, -3)), (66, (7, 8)),...","[[The, first, thing, we, do, is, approach, the..."
2,"(832, 834)",42,"[(42, (832, 834))]",[],"[(42, (24, 26))]","[[They, feel, more, protected, if, we, are, th..."
3,"(857, 859)",44,"[(46, (941, 942)), (45, (889, 890))]","[(44, (857, 859))]","[(46, (28, 29)), (45, (1, 2))]","[[His, arrest, and, the, distinctive, red, cro..."
5,"(1043, 1045)",52,"[(75, (1400, 1401)), (74, (1400, 1401)), (73, ...",[],"[(75, (-4, -3)), (74, (-3, -2)), (73, (6, 7)),...","[[The, kids, appreciate, our, work, because, n..."


In [90]:
s

[['SANTIAGO', ',', 'Chile', '—'],
 ['They',
  'appear',
  'at',
  'the',
  'student',
  'demonstrations',
  'that',
  'are',
  'once',
  'again',
  'filling',
  'the',
  'streets',
  'and',
  'occupying',
  'the',
  'schools',
  'of',
  'Santiago',
  ',',
  'and',
  'at',
  'the',
  'hospitals',
  'and',
  'police',
  'stations',
  'where',
  'the',
  'fallout',
  'lands',
  'afterward',
  ':',
  'small',
  'troops',
  'of',
  'observers',
  'in',
  'blue',
  'or',
  'white',
  'helmets',
  ',',
  'armed',
  'with',
  'notebooks',
  ',',
  'cameras',
  ',',
  'voice',
  'recorders',
  'and',
  'gas',
  'masks',
  '.'],
 ['They',
  'are',
  'not',
  'there',
  'to',
  'join',
  'the',
  'protests',
  'or',
  'interfere',
  ',',
  'only',
  'to',
  'monitor',
  'and',
  'record',
  'what',
  'happens',
  'when',
  'the',
  'police',
  'crack',
  'down',
  'on',
  'the',
  'protests',
  '—',
  'as',
  'they',
  'have',
  'done',
  'with',
  'increased',
  'violence',
  'this',
  'year',
 

In [ ]:
## todo:

## 1. swap head spans with other things: "anonymous official", "[PERSON]", etc.
## 2. remove quotes
## 3. negatively sample
## 4. re-use sentence-level BERT classification architecture to classify each sentence.

## once each sentence is classified, find a way to extract the set of NEs, terms, etc. from the article
## use BERT to do quote attribution
## 